In [1]:
from __future__ import division, unicode_literals
import numpy as np
from bqplot import Axis, Bars, Figure, LinearScale, Lines, LogScale, Scatter, Tooltip
from collections import namedtuple
from ipywidgets import Layout
from scipy.ndimage.filters import convolve1d

import warnings
warnings.filterwarnings('ignore')

In [2]:
import bqplot
print(bqplot.__version__)
import ipywidgets
print(ipywidgets.__version__)

0.9.0.b7
5.2.2


In [3]:
ACES = np.logspace(np.log10(1e-5), np.log10(1e5), 1000)
XMID = 10**0
XMAX = 10**4
XMIN = 10**-4
YMID = 10**0
YMAX = 10**4
YMIN = 10**-4
LO_SLOPE = 0.25
HI_SLOPE = 0.25
MID_SLOPE = 1.5
NKNOTSLOW = 4
NKNOTSHIGH = 4
COEFSLOW = np.array(
    [0,
     0,
     np.log10(YMIN) + 0.25 *
     (np.log10(YMID) - np.log10(YMIN)),
     0,
     0])
COEFSHIGH = np.array(
    [0,
     0,
     np.log10(YMID) + 0.75 *
     (np.log10(YMAX) - np.log10(YMID)),
     0,
     0])


def system_tonescale_segmented(aces=ACES,
                               xmid=XMID,
                               xmax=XMAX,
                               xmin=XMIN,
                               ymid=YMID,
                               ymax=YMAX,
                               ymin=YMIN,
                               lo_slope=LO_SLOPE,
                               hi_slope=HI_SLOPE,
                               mid_slope=MID_SLOPE,
                               nKnotsLow=NKNOTSLOW,
                               nKnotsHigh=NKNOTSHIGH,
                               coefsLow=COEFSLOW,
                               coefsHigh=COEFSHIGH):
    aces = np.asarray(aces)

    coefsLow = COEFSLOW if coefsLow is None else coefsLow
    coefsHigh = COEFSHIGH if coefsLow is None else coefsHigh

    # Textbook monomial to basis-function conversion matrix
    m = np.array([[0.5, -1.0,  0.5], [-1.0,  1.0,  0.0], [0.5,  0.5,  0.0]])

    # B-spline coefficients are are located at the midpoints of each knot
    # interval.

    # First B-spline
    # Calculate values to be used in application of the B-spline s
    knot_xLow = np.linspace(np.log10(xmin), np.log10(xmid), nKnotsLow)
    knotIncLow = (knot_xLow[-1] - knot_xLow[0]) / (nKnotsLow - 1)

    knot_xHigh = np.linspace(np.log10(xmid), np.log10(xmax), nKnotsHigh)
    knotIncHigh = (knot_xHigh[-1] - knot_xHigh[0]) / (nKnotsHigh - 1)

    # This block can be commented out if co/kn outputs are not used
    coef_xLow = np.linspace(np.log10(xmin) - 0.5 * knotIncLow,
                            np.log10(xmid) + 0.5 * knotIncLow, nKnotsLow + 1)
    coef_xHigh = np.linspace(np.log10(
        xmid) - 0.5 * knotIncHigh, np.log10(xmax) + 0.5 * knotIncHigh, nKnotsHigh + 1)

    coefsLow[0] = ((lo_slope * coef_xLow[0]) +
                   (np.log10(ymin) - lo_slope * knot_xLow[0]))
    coefsLow[1] = ((lo_slope * coef_xLow[1]) +
                   (np.log10(ymin) - lo_slope * knot_xLow[0]))

    coefsLow[-2] = ((mid_slope * coef_xLow[-2]) +
                    (np.log10(ymid) - mid_slope * knot_xLow[-1]))
    coefsLow[-1] = ((mid_slope * coef_xLow[-1]) +
                    (np.log10(ymid) - mid_slope * knot_xLow[-1]))

    coefsHigh[0] = ((mid_slope * coef_xHigh[0]) +
                    (np.log10(ymid) - mid_slope * knot_xHigh[0]))
    coefsHigh[1] = ((mid_slope * coef_xHigh[1]) +
                    (np.log10(ymid) - mid_slope * knot_xHigh[0]))

    coefsHigh[-2] = ((hi_slope * coef_xHigh[-2]) +
                     (np.log10(ymax) - hi_slope * knot_xHigh[-1]))
    coefsHigh[-1] = ((hi_slope * coef_xHigh[-1]) +
                     (np.log10(ymax) - hi_slope * knot_xHigh[-1]))

    # Parameters calculated from editable parameters:
    # Knot_dens is density of the spline at the knots
    knot_yLow = convolve1d(coefsLow, [0.5, 0.5])
    knot_yHigh = convolve1d(coefsHigh, [0.5, 0.5])

    # Calculations

    # Check for negatives or zero before taking the log. If negative or zero,
    # set to ACESMIN
    aces[aces <= 0] = 2**-14

    # Allocate an array for the output values
    logOut = np.zeros(np.shape(aces))

    # Take log of linear input
    logAces = np.log10(aces)

    indexLow = (logAces <= np.log10(xmin))
    indexLowHalf = ((logAces > np.log10(xmin)) & (logAces < np.log10(xmid)))
    indexHighHalf = ((logAces >= np.log10(xmid)) & (logAces < np.log10(xmax)))
    indexHigh = logAces >= np.log10(xmax)

    # For values outside the B-spline range, apply linear extension
    logOut[indexLow] = (logAces[indexLow] * lo_slope +
                        (np.log10(ymin) - lo_slope * knot_xLow[0]))

    # For typical OUT values apply the B-spline curve.
    if np.sum(indexLowHalf.astype(int)) > 0:
        knot_coord = ((nKnotsLow - 1) *
                      (logAces[indexLowHalf] - knot_xLow[0]) /
                      (knot_xLow[-1] - knot_xLow[0]))
        jLow = knot_coord.astype(int)
        tLow = knot_coord - jLow

        cfLow = np.array(coefsLow[jLow])
        cfLow = np.vstack((cfLow, coefsLow[jLow + 1]))
        cfLow = np.vstack((cfLow, coefsLow[jLow + 2]))

        monomialsLow = np.array([tLow * tLow, tLow, np.ones(np.size(tLow))])
        basisLow = np.dot(m, cfLow)
        logOut[indexLowHalf] = np.sum(np.multiply(monomialsLow, basisLow), 0)

    # Do the other B-spline for indexHigh
    if np.sum(indexHighHalf.astype(int)) > 0:
        knot_coord = ((nKnotsHigh - 1) *
                      (logAces[indexHighHalf] - knot_xHigh[0]) /
                      (knot_xHigh[-1] - knot_xHigh[0]))
        jHigh = knot_coord.astype(int)
        tHigh = knot_coord - jHigh

        cfHigh = np.array(coefsHigh[jHigh])
        cfHigh = np.vstack((cfHigh, coefsHigh[jHigh + 1]))
        cfHigh = np.vstack((cfHigh, coefsHigh[jHigh + 2]))

        monomialsHigh = np.array(
            [tHigh * tHigh, tHigh, np.ones(np.size(tHigh))])
        basisHigh = np.dot(m, cfHigh)
        logOut[indexHighHalf] = np.sum(
            np.multiply(monomialsHigh, basisHigh), 0)

    # For values outside the B-spline range, apply linear extension
    logOut[indexHigh] = (logAces[indexHigh] * hi_slope +
                         (np.log10(ymax) - hi_slope * knot_xHigh[-1]))

    kn = {'x': np.hstack((knot_xLow, knot_xHigh)),
          'y': np.hstack((knot_yLow, knot_yHigh))}

    co = {'x': np.hstack((coef_xLow, coef_xHigh)),
          'y': np.hstack((coefsLow, coefsHigh))}

    out = 10**logOut

    return out, kn, co

In [4]:
SPLINE_SCALE_X = LogScale(min=1e-5, max=1e5)
SPLINE_SCALE_Y = LogScale(min=0.00001, max=100000)

SPLINE_AXIS_X = Axis(scale=SPLINE_SCALE_X, grid_lines='solid', label='X')
SPLINE_AXIS_Y = Axis(scale=SPLINE_SCALE_Y, grid_lines='solid', label='Y', orientation='vertical')

SPLINE_LINE = Lines(
    x=ACES, 
    y=system_tonescale_segmented(
        ACES, XMID, XMAX, XMIN, YMID, YMAX, YMIN, 
        LO_SLOPE, HI_SLOPE, MID_SLOPE, 
        NKNOTSLOW, NKNOTSHIGH, COEFSLOW, COEFSHIGH)[0], 
    scales={'x': SPLINE_SCALE_X, 'y': SPLINE_SCALE_Y})

SPLINE_SCATTER_TOOLTIP = Tooltip(fields=['x', 'y'], formats=['.5f', '.5f'], labels=['x', 'y'])
SPLINE_SCATTER = Scatter(
    x=[XMIN, XMID, XMAX, XMIN, XMID, XMAX],
    y=[YMIN, YMID, YMAX, LO_SLOPE, MID_SLOPE, HI_SLOPE],
    colors=['deepskyblue', 'deepskyblue', 'deepskyblue', 'red', 'red', 'red'],
    default_opacities=[0.85, 0.85, 0.85, 0.25, 0.25, 0.25],
    scales={'x': SPLINE_SCALE_X, 'y': SPLINE_SCALE_Y},
    tooltip=SPLINE_SCATTER_TOOLTIP,
    enable_move=True,
    update_on_move=True)

Figure(marks=[SPLINE_LINE, SPLINE_SCATTER], 
       axes=[SPLINE_AXIS_X, SPLINE_AXIS_Y],
       layout=Layout(min_width='960px', min_height='540px'),
       title='Spline Interactive')

The installed widget Javascript is the wrong version.


In [5]:
def on_spline_scatter_drag(name, value):
    SPLINE_LINE.y = system_tonescale_segmented(
        ACES, SPLINE_SCATTER.x[1], SPLINE_SCATTER.x[2], SPLINE_SCATTER.x[0], SPLINE_SCATTER.y[1], SPLINE_SCATTER.y[2], SPLINE_SCATTER.y[0],
        np.log10(SPLINE_SCATTER.y[3]) - np.log10(LO_SLOPE) + LO_SLOPE, 
        np.log10(SPLINE_SCATTER.y[5]) - np.log10(HI_SLOPE) + HI_SLOPE, 
        np.log10(SPLINE_SCATTER.y[4]) - np.log10(MID_SLOPE) + MID_SLOPE,
        NKNOTSLOW, NKNOTSHIGH, COEFSLOW, COEFSHIGH)[0]
    
SPLINE_SCATTER.on_drag_start(on_spline_scatter_drag)
SPLINE_SCATTER.on_drag(on_spline_scatter_drag)
SPLINE_SCATTER.on_drag_end(on_spline_scatter_drag)